In [1]:
import pandas as pd
import os
import re

In [2]:
def parse_log_file(log_file_path):
    model_size = None
    inference_time = None
    
    with open(log_file_path, 'r') as file:
        for line in file:
            # Extract model size
            size_match = re.search(r'Size \(MB\): ([\d\.]+)', line)
            if size_match:
                model_size = float(size_match.group(1))
            
            # Extract average inference time
            time_match = re.search(r'Average inference time: ([\d\.]+)', line)
            if time_match:
                inference_time = float(time_match.group(1))
                
    return model_size, inference_time

In [9]:
df = pd.read_csv('../../analysis/experiment_values.csv')
df.head()

,Task,Compression Method,Parameter,Value
0,Defect Prediction,No One,Avg Inference Time CUDA,0.010
1,Defect Prediction,Distillation (distilbert),Avg Inference Time CUDA,0.005
2,Defect Prediction,Quantization (quanto-qfloat8),Avg Inference Time CUDA,0.021
3,Defect Prediction,Quantization (quanto-qint8),Avg Inference Time CUDA,0.019
4,Defect Prediction,Quantization (quanto-qint4),Avg Inference Time CUDA,0.030


In [10]:
for logs in os.listdir('logs_t5'):
  model_size, inf_time = parse_log_file(f'logs_t5/{logs}')
  task = 'Code Summarization T5'
  if 'prune4' in logs:
    compression = 'Pruning 0.4'
  elif 'prune6' in logs:
    compression = 'Pruning 0.6'
  elif 'prune' in logs:
    compression = 'Pruning 0.2'
  elif 'quantf8' in logs:
    compression = 'Quantization (quanto-qfloat8)'
  elif 'quant4' in logs:
    compression = 'Quantization (quanto-qint4)'
  elif 'quant' in logs:
    compression = 'Quantization (quanto-qint8)'
  else:
    compression = 'No One'
  if 'cuda' in logs:
    df = pd.concat([df, pd.DataFrame({
      'Task': [task], 'Compression Method': [compression], 'Parameter': ["Avg Inference Time CUDA"], 'Value': [inf_time]})])  
  else:
    df = pd.concat([df, pd.DataFrame({
      'Task': [task], 'Compression Method': [compression], 'Parameter': ["Avg Inference Time CPU"], 'Value': [inf_time]})])
    df = pd.concat([df, pd.DataFrame({
      'Task': [task], 'Compression Method': [compression], 'Parameter': ["Model Size"], 'Value': [model_size]})])
 

In [11]:
df.tail(n=25)

,Task,Compression Method,Parameter,Value
280,Defect Prediction Graph,Pruning 0.4,MCC,0.221497
281,Defect Prediction Graph,Pruning 0.2,Accuracy,0.621157
282,Defect Prediction Graph,Pruning 0.2,F1,0.552142
283,Defect Prediction Graph,Pruning 0.2,MCC,0.230633
0,Code Summarization T5,No One,Avg Inference Time CPU,12.388410
0,Code Summarization T5,No One,Model Size,891.644607
0,Code Summarization T5,Quantization (quanto-qint8),Avg Inference Time CUDA,2.273070
0,Code Summarization T5,Pruning 0.6,Avg Inference Time CPU,11.792607
0,Code Summarization T5,Pruning 0.6,Model Size,891.644607
0,Code Summarization T5,Pruning 0.4,Avg Inference Time CPU,12.328668


In [12]:
df.to_csv('../../analysis/experiment_values.csv', index=False)